In [ ]:
!pip3 install pyspark==3.0.1

     |████████████████████████████████| 204.2 MB 35 kB/s 
     |████████████████████████████████| 198 kB 67.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=d871c9f2dda79acd7eb96f362aed3606b30e07e9ddfa00c8c28adfc27cc75713
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
!sudo apt update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15 http://ppa

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz  #install Apache Spark
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

import os 
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.0.1-bin-hadoop2.7'

In [ ]:
!pip install findspark

# *Importing libraries and modules and loading the data:*

In [ ]:
import os
import pandas as pd
import numpy as np
from termcolor import colored
import gdown

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f

import findspark
findspark.init()

In [ ]:
!gdown --id 1oDcTYWLry7jo3nij3TGxFOGHVP6-LjPt

Downloading...
From: https://drive.google.com/uc?id=1oDcTYWLry7jo3nij3TGxFOGHVP6-LjPt
To: /content/2019-Oct.csv.zip
100% 1.73G/1.73G [00:12<00:00, 136MB/s]


In [ ]:
!unzip "/content/2019-Oct.csv.zip" -d "/content"

Archive:  /content/2019-Oct.csv.zip
  inflating: /content/2019-Oct.csv   


In [ ]:
new_session = SparkSession.builder.master("local").appName("CIE_427_Final_Project").getOrCreate()

In [ ]:
dataset = new_session.read.format("csv").option("delimiter", ",").option("ESCAPE quote", '"'). option("header",True).load("/content/2019-Oct.csv")
dataset.show(10)

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    null| 543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:...|      view|   1004237|205301355563188265

# *Project requirments:*

## *$1^{st}$ Requirment:*

### top brands 

In [ ]:
top_brands = dataset.groupBy(["brand","event_type"]).count().sort('count', ascending = False).na.drop('any')
top_brands.cache()

In [ ]:
top_brands.show(10) # Top 10 brands.

+-------+----------+-------+
|  brand|event_type|  count|
+-------+----------+-------+
|samsung|      view|4806630|
|  apple|      view|3770597|
| xiaomi|      view|2922650|
| huawei|      view|1045572|
|lucente|      view| 644283|
|  bosch|      view| 544230|
|     lg|      view| 540131|
|   oppo|      view| 451449|
|   sony|      view| 439855|
|   acer|      view| 411871|
+-------+----------+-------+
only showing top 10 rows



### top viewed brands

In [ ]:
top_viewed_brands = top_brands.filter(top_brands.event_type == 'view') 
top_viewed_brands.cache()

In [ ]:
top_viewed_brands.show(10) # Top 10 viewed brands.

### top purchased brands

In [ ]:
top_purchased_brands = top_viewed_brands.filter(top_viewed_brands.event_type == 'purchase') 
top_purchased_brands.cache()

In [ ]:
top_purchased_brands.show(10) # Top 10 purchased brands.

+--------+----------+------+
|   brand|event_type| count|
+--------+----------+------+
| samsung|  purchase|172896|
|   apple|  purchase|142873|
|  xiaomi|  purchase| 56616|
|  huawei|  purchase| 23501|
| lucente|  purchase| 11578|
|    oppo|  purchase| 10891|
|cordiant|  purchase| 10551|
|      lg|  purchase|  8727|
|    acer|  purchase|  6882|
|    sony|  purchase|  6729|
+--------+----------+------+
only showing top 10 rows



## *$2^{nd}$ Requirment:*

In [ ]:
# Get names of the 10 top purchased brands.
top_brands = list()
top_brands_count = 10

for i in range(top_brands_count):
  brand_name = top_purchased_brands.collect()[i]['brand']
  top_brands.append(brand_name)

print(colored('The top 10 brands are ', 'red') + top_brands[0] + ', ' + top_brands[1] + ', ' + top_brands[2] + ', ' + top_brands[3] + ', ' + top_brands[4] + ', ' + top_brands[5] + ', ' + top_brands[6] + ', ' + top_brands[7] + ', ' + top_brands[8] + ', ' + top_brands[9])

The top 10 brands are samsung, apple, xiaomi, huawei, lucente, oppo, cordiant, lg, acer, sony


In [ ]:
# Get the most purchased item per brand.
for i in range(top_brands_count):
  brands_items = dataset.groupBy(["brand", "category_code"]).count().filter(dataset.brand == top_brands[i]).sort(['count'], ascending = False).na.drop('any').collect()
  
  if brands_items:
    most_purchased_item = brands_items[0]["category_code"].split('.')
    most_purchased_item = ' '.join(most_purchased_item[1:])
  
    print(colored('Brand: ','red') + brands_items[0]["brand"] + colored(', Most purchased item: ','red') + most_purchased_item)
    

Brand: samsung, Most purchased item: smartphone
Brand: apple, Most purchased item: smartphone
Brand: xiaomi, Most purchased item: smartphone
Brand: huawei, Most purchased item: smartphone
Brand: oppo, Most purchased item: smartphone
Brand: lg, Most purchased item: kitchen washer
Brand: acer, Most purchased item: notebook
Brand: sony, Most purchased item: smartphone


## *$3^{rd}$ Requirment:*

In [ ]:
most_purchasing_users = dataset.groupBy(["user_id","event_type"]).count().filter(dataset.event_type == 'purchase').sort('count', ascending = False).na.drop('any')

most_purchasing_users.cache()
most_purchasing_users.show(20) #Top 20 purchasing users.

+---------+----------+-----+
|  user_id|event_type|count|
+---------+----------+-----+
|512386086|  purchase|  322|
|523974502|  purchase|  291|
|513320236|  purchase|  242|
|517728689|  purchase|  204|
|543312954|  purchase|  193|
|513117637|  purchase|  185|
|519267944|  purchase|  183|
|530834332|  purchase|  170|
|541510103|  purchase|  165|
|549109608|  purchase|  154|
|551557821|  purchase|  154|
|513322839|  purchase|  146|
|546967331|  purchase|  141|
|556747652|  purchase|  139|
|519322381|  purchase|  136|
|558420737|  purchase|  132|
|547330965|  purchase|  131|
|547101738|  purchase|  130|
|525351013|  purchase|  129|
|551361102|  purchase|  127|
+---------+----------+-----+
only showing top 20 rows



## *$4^{th}$ Requirment:*

In [ ]:
# Get IDs of the 20 top purchasing users. 
top_users_id = list()
top_users_count = 20

for i in range(top_users_count):
  user_id = most_purchasing_users.collect()[i]['user_id']
  top_users_id.append(user_id)
top_users_id  

['512386086',
 '523974502',
 '513320236',
 '517728689',
 '543312954',
 '513117637',
 '519267944',
 '530834332',
 '541510103',
 '549109608',
 '551557821',
 '513322839',
 '546967331',
 '556747652',
 '519322381',
 '558420737',
 '547330965',
 '547101738',
 '525351013',
 '551361102']

In [ ]:
# Get most purchased category per user.
for i in range(len(top_users_id)):
  user_categories = []

  purchased_categories_by_user = dataset.groupBy(["user_id", "category_code"]).count().filter(dataset.user_id == top_users_id[i]).sort(['count'], ascending = False).na.drop('any').collect()
  
  for j in range(len(purchased_categories_by_user)):
    user_category = purchased_categories_by_user[j]["category_code"].split('.')[0]
    user_categories.append(user_category)

  if user_categories:
    most_purchased_category_by_user = max(set(user_categories), key = user_categories.count)
    print(colored('User ID: ','red') + top_users_id[i] + ',  ' + colored('Most purchased category:','red') + most_purchased_category_by_user)
    

User ID: 512386086,  Most purchased category:appliances
User ID: 523974502,  Most purchased category:appliances
User ID: 513320236,  Most purchased category:appliances
User ID: 517728689,  Most purchased category:apparel
User ID: 543312954,  Most purchased category:appliances
User ID: 513117637,  Most purchased category:electronics
User ID: 519267944,  Most purchased category:electronics
User ID: 530834332,  Most purchased category:electronics
User ID: 541510103,  Most purchased category:electronics
User ID: 549109608,  Most purchased category:electronics
User ID: 551557821,  Most purchased category:electronics
User ID: 513322839,  Most purchased category:electronics
User ID: 546967331,  Most purchased category:electronics
User ID: 556747652,  Most purchased category:appliances
User ID: 519322381,  Most purchased category:electronics
User ID: 558420737,  Most purchased category:electronics
User ID: 547330965,  Most purchased category:electronics
User ID: 547101738,  Most purchased cate

## *$5^{th}$ Requirment:*

In [ ]:
added_to_cart = dataset.groupBy(["brand","event_type"]).count().filter(dataset.event_type == 'cart').sort('count', ascending = False).na.drop('any')
added_to_cart.show(10) # Top 10 brands added to cart.

+--------+----------+------+
|   brand|event_type| count|
+--------+----------+------+
| samsung|      cart|303249|
|   apple|      cart|209084|
|  xiaomi|      cart|104497|
|  huawei|      cart| 42132|
|    oppo|      cart| 20547|
|      lg|      cart| 13546|
|    sony|      cart| 10060|
|    acer|      cart|  9400|
|elenberg|      cart|  8598|
|   bosch|      cart|  7155|
+--------+----------+------+
only showing top 10 rows



In [ ]:
removed_from_cart = dataset.groupBy(["brand","event_type"]).count().filter(dataset.event_type == 'remove_from_cart').sort('count', ascending = False)
removed_from_cart.show(10) # Top 10 brands removed from cart.

+-----+----------+-----+
|brand|event_type|count|
+-----+----------+-----+
+-----+----------+-----+



## *$6^{th}$ Requirment:*

In [ ]:
category_names_dataset = dataset.withColumn('category_name', f.split('category_code', '\.').getItem(0))

In [ ]:
category_names_dataset.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|category_name|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|         null|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|   appliances|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    null| 543.10|519107250|566511c2-e2e3-422...|    furniture|
|2019-10-01 00:00:...|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|

In [ ]:
viewed_categories = category_names_dataset.groupBy(["category_name", "event_type"]).count().filter(category_names_dataset.event_type == 'view').sort('count', ascending = False).na.drop('any')
viewed_categories.show(10) # Top 10 most viewed categories.

+-------------+----------+--------+
|category_name|event_type|   count|
+-------------+----------+--------+
|  electronics|      view|15038390|
|   appliances|      view| 4799936|
|    computers|      view| 2268928|
|      apparel|      view| 1534896|
|    furniture|      view| 1236452|
|         auto|      view|  993428|
| construction|      view|  712690|
|         kids|      view|  512485|
|  accessories|      view|  236102|
|        sport|      view|  174324|
+-------------+----------+--------+
only showing top 10 rows



# *Machine learning requirement:*

### import libraries

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [ ]:
df_pur_cart = dataset.filter((dataset.event_type == 'purchase') | (dataset.event_type == 'cart') )

In [ ]:
df_dis=df_pur_cart.dropDuplicates(['event_type', 'product_id','price','user_id','user_session'])

In [ ]:
from pyspark.sql import functions as f
df_addcol=df_dis.withColumn('is_purchased', f.when(f.col('event_type') == 'purchase', 1).otherwise(0))

In [ ]:
df_fin2=df_addcol.select('event_time','brand','category_code','price','is_purchased')

In [ ]:
(training_data, test_data) = df_fin2.randomSplit([0.8,0.2], seed = 2022)

In [ ]:
# define stage 1: transform the column feature_2 to numeric
stage_1 = StringIndexer(inputCol= 'event_time', outputCol= 'event_time_index')
stage_1.setHandleInvalid("skip")
# define stage 2: transform the column feature_3 to numeric
stage_2 = StringIndexer(inputCol= 'brand', outputCol= 'brand_index').setHandleInvalid("skip")

stage_3 = StringIndexer(inputCol= 'category_code', outputCol= 'category_code_index').setHandleInvalid("skip")

stage_4 = StringIndexer(inputCol= 'price', outputCol= 'price_index').setHandleInvalid("skip")


# define stage 3: one hot encode the numeric versions of feature 2 and 3 generated from stage 1 and stage 2
stage_5 = OneHotEncoder(inputCols=[stage_1.getOutputCol(), stage_2.getOutputCol(),stage_3.getOutputCol(),stage_4.getOutputCol()], 
                                 outputCols= ['event_time_encoded', 'brand_encoded','category_code_encoded','price_encoded'])
# define stage 4: create a vector of all the features required to train the logistic regression model 
stage_6 = VectorAssembler(inputCols=['event_time_encoded', 'brand_encoded','category_code_encoded','price_encoded'],
                          outputCol='features')
# define stage 5: logistic regression model                          
stage_7 = LogisticRegression(featuresCol='features',labelCol='is_purchased')

# setup the pipeline
regression_pipeline = Pipeline(stages= [stage_1, stage_2, stage_3, stage_4, stage_5,stage_6,stage_7])

# fit the pipeline for the trainind data
model = regression_pipeline.fit(training_data)
# transform the data
training_data = model.transform(training_data)

# view some of the columns generated
training_data.select('features', 'is_purchased', 'rawPrediction', 'probability', 'prediction').show()

TO display the ROC curve 

In [ ]:
training_summary = model.stages[-1].summary
roc = training_summary.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False +ve Rate')
plt.xlabel('True +ve Rate')
plt.title('ROC Curve')
plt.show()
print('Training ROC: ' + str(training_summary.areaUnderROC))